In [ ]:
%%capture

from IPython.utils import io
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

In [ ]:
# https://www.kaggle.com/discussions/product-feedback/508758
! rm -rf /opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info
# do above to get 
!pip install -qq llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface


In [ ]:
!curl https://ollama.ai/install.sh | sh

In [ ]:
!ollama

In [ ]:
import subprocess
import time

# Start ollama as a backrgound process
command = "nohup ollama serve&"

# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command,
                            shell=True,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)
print("Process ID:", process.pid)
# Let's use fly.io resources
#!OLLAMA_HOST=https://ollama-demo.fly.dev:443
time.sleep(5)  # Makes Python wait for 5 seconds

In [ ]:
!ollama list

In [ ]:
!ollama pull llama3

### Basic Completion and Chat

Call complete with a prompt

In [ ]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3", request_timeout=120.0)
resp = llm.complete("Tell me about Ulysses by James Joyce?")
print(resp)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
from llama_index.core import SimpleDirectoryReader

ulysses = SimpleDirectoryReader(input_files=["/kaggle/input/ulysses/ulysses.txt"]).load_data()


### Basic RAG (Vector Search)

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(ulysses)
query_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
response = query_engine.query("What is Calypso episode about?")
print(str(response))

### Basic RAG (Summarization)

In [ ]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(ulysses)
summary_engine = summary_index.as_query_engine()

In [ ]:
response = summary_engine.query(
    "Give me a short summary of the “Calypso” episode"
)
print(str(response))